In [2]:
import math
from matplotlib import pyplot as plt
import numpy as np
from cmath import sqrt

# Ejercicio 3

Suponga que un viajante tiene que visitar $n$ ciudades en el menor tiempo posible. Considere una matriz $D$ de tamaño $n x n$ cuyos elementos $d_{pq}$ denotan la distancia entre cada par de ciudades $(p, q)$. Se define un recorrido como una trayectoria cerrada que visita cada ciudad una y sólo una vez (a excepción de la ciudad de partida, a la cual debe regresar). El problema consiste en hallar el recorrido de mínima longitud.

Implemente el algoritmo de sistema de hormigas y utilícelo para resolver el problema del agente viajero considerando los datos proporcionados en el archivo `gr17.csv`. Analice el efecto de la tasa de evaporación $(ρ)$ y de la cantidad de feromona depositada $(\tau)$ sobre los resultados de la búsqueda. Para esto último compare el desempeño del algoritmo empleando los métodos global, local y uniforme para depósito de feromonas. Realice varias corridas con cada configuración experimental y considere el tiempo de búsqueda y la longitud de los caminos encontrados como medidas para comparar el desempeño. Construya una tabla comparativa con los resultados obtenidos.

In [3]:
# Matriz de distancias entre ciudades
distanceMatrix = np.genfromtxt('gtp4datos/gr17.csv', delimiter=',')
# relleno la diagonal con infinito para que no haya división por cero
np.fill_diagonal(distanceMatrix, np.inf)

In [4]:
import random as rn
import numpy as np
from numpy.random import choice as np_choice

class AntColony(object):

    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=1):
        """
        Args:
            distances (2D numpy.array): Square matrix of distances. Diagonal is assumed to be np.inf.
            n_ants (int): Number of ants running per iteration
            n_best (int): Number of best ants who deposit pheromone
            n_iteration (int): Number of iterations
            decay (float): Rate it which pheromone decays. The pheromone value is multiplied by decay, so 0.95 will lead to decay, 0.5 to much faster decay.
            alpha (int or float): exponenet on pheromone, higher alpha gives pheromone more weight. Default=1
            beta (int or float): exponent on distance, higher beta give distance more weight. Default=1
        Example:
            ant_colony = AntColony(german_distances, 100, 20, 2000, 0.95, alpha=1, beta=2)          
        """
        self.distances  = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.all_inds = range(len(distances))
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta

    def run(self):
        shortest_path = None
        all_time_shortest_path = ("placeholder", np.inf)
        for i in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheronome(all_paths, self.n_best, shortest_path=shortest_path)
            shortest_path = min(all_paths, key=lambda x: x[1])
            print (shortest_path)
            if shortest_path[1] < all_time_shortest_path[1]:
                all_time_shortest_path = shortest_path            
            self.pheromone = self.pheromone * self.decay            
        return all_time_shortest_path

    def spread_pheronome(self, all_paths, n_best, shortest_path):
        sorted_paths = sorted(all_paths, key=lambda x: x[1])
        for path, dist in sorted_paths[:n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / self.distances[move]

    def gen_path_dist(self, path):
        total_dist = 0
        for ele in path:
            total_dist += self.distances[ele]
        return total_dist

    def gen_all_paths(self):
        all_paths = []
        for i in range(self.n_ants):
            path = self.gen_path(0)
            all_paths.append((path, self.gen_path_dist(path)))
        return all_paths

    def gen_path(self, start):
        path = []
        visited = []
        visited.append(start)
        prev = start
        for i in range(len(self.distances) - 1):
            move = self.pick_move(self.pheromone[prev], self.distances[prev], visited)
            path.append((prev, move))
            prev = move
            visited.append(move)
        path.append((prev, start)) # going back to where we started    
        return path

    def pick_move(self, pheromone, dist, visited):
        pheromone = np.copy(pheromone)
        print('visitado:', visited)
        pheromone[visited] = 0

        row = pheromone ** self.alpha * (( 1.0 / dist) ** self.beta)

        norm_row = row / row.sum()
        move = np_choice(self.all_inds, 1, p=norm_row)[0]
        return move

In [6]:
ant_colony = AntColony(distanceMatrix, 100, 20, 10, 0.95, alpha=1, beta=2)

ant_colony.run()

visitado: [0]
visitado: [0, 3]
visitado: [0, 3, 12]
visitado: [0, 3, 12, 16]
visitado: [0, 3, 12, 16, 13]
visitado: [0, 3, 12, 16, 13, 10]
visitado: [0, 3, 12, 16, 13, 10, 9]
visitado: [0, 3, 12, 16, 13, 10, 9, 1]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6, 7]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6, 7, 5]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6, 7, 5, 14]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6, 7, 5, 14, 2]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6, 7, 5, 14, 2, 4]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6, 7, 5, 14, 2, 4, 8]
visitado: [0, 3, 12, 16, 13, 10, 9, 1, 6, 7, 5, 14, 2, 4, 8, 11]
visitado: [0]
visitado: [0, 14]
visitado: [0, 14, 5]
visitado: [0, 14, 5, 7]
visitado: [0, 14, 5, 7, 16]
visitado: [0, 14, 5, 7, 16, 6]
visitado: [0, 14, 5, 7, 16, 6, 12]
visitado: [0, 14, 5, 7, 16, 6, 12, 3]
visitado: [0, 14, 5, 7, 16, 6, 12, 3, 13]
visitado: [0, 14, 5, 7, 16, 6, 12, 3, 13, 2]
visitado: [0, 14, 5, 7, 16, 6, 12, 3, 13, 2, 10]
visitado: [0, 1

([(0, 3),
  (3, 12),
  (12, 6),
  (6, 7),
  (7, 16),
  (16, 5),
  (5, 13),
  (13, 14),
  (14, 2),
  (2, 10),
  (10, 9),
  (9, 1),
  (1, 4),
  (4, 8),
  (8, 11),
  (11, 15),
  (15, 0)],
 2096.0)